In [33]:
import numpy as np
import pandas as pd
import numpy.linalg as la
from scipy.linalg import solve_triangular
from typing import Callable


### FAIPA

In [34]:
def line_search(f: Callable, df: Callable, g: Callable, x: np.ndarray, eta: float, d: np.ndarray, d3: np.ndarray, bar_lambda: np.ndarray, nu: float, m: int):
    """
    Line-Search
    -----------
    Compute t which is the step length, the first number of the sequence {1, v, v2, v3 ,...} that satisfying the following conditions

    Parameters
    ----------
    eta in (0, 1)
    d direction
    bar_lambda lagrangian multiplier
    nu  in (0, 1)

    Return
    ------
    The step length
    """
    t = 1
    while True:
        x_td = x + t * (d + t * d3)
        if f(x_td) <= f(x) + t * eta * d.dot(df(x)):
            g_x = g(x)
            g_x_td = g(x_td)
            if all((g_x_td[k] < 0 and bar_lambda[k] >= 0) or (g_x_td[k] <= g_x[k] and bar_lambda[k] < 0) for k in range(m)):
                break
        t = t * nu
    return t


**Example**

\begin{align*}
\min_{\mathbf x}\quad&2x^2+y^2\\
\text{s.t}\quad&4x^2+y^2-2 \le 0\\ 
&4x+y+1\le0
\end{align*}

In [35]:

def FAIPA(f: Callable, df: Callable, g: Callable, dg: Callable, x: np.ndarray, tol: float, xi: float, eta: float, phi: float, nu: float, m: int, MAX_ITER=100):
    lam = np.ones(m)
    w = np.ones(m)
    n = len(x)
    iter = 0
    t = 1
    B = np.identity(n)
    Z = np.diag(lam)
    while iter < MAX_ITER:
        g_gradient = dg(x)
        f_gradient = df(x)
        G = np.diag(g(x))
        N = np.r_[np.c_[B, g_gradient],
                  np.c_[Z @ g_gradient.T, G]]
        Q, R = la.qr(N)
        z1 = Q.T @ np.r_[-f_gradient,
                         np.zeros(m)]
        # solve system 1
        x_1 = solve_triangular(R, z1)
        d_a = x_1[:n]
        lambda_a = x_1[n:]
        error = la.norm(d_a)
        if error < tol:
            return x
        z2 = Q.T @ np.r_[np.zeros(n),
                         - Z @ w]
        # solve system 2
        x_2 = solve_triangular(R, z2)
        d_b = x_2[:n]
        lambda_b = x_2[n:]
        a = phi * d_a.dot(d_a)
        if (d2f_grad := d_b.dot(f_gradient)) > 0:
            b = (xi - 1) * d_a.dot(f_gradient) / d2f_grad
            rho = min(a, b)
        else:
            rho = a
        d = d_a + rho * d_b
        bar_lambda = lambda_a + rho * lambda_b
        omega = f(x + d) - f(x) - d.dot(g_gradient)
        z3 = Q.T @ np.r_[np.zeros(n),
                         - Z @ omega]
        # solve system 3
        x_3 = solve_triangular(R, z3)
        d_c = x_3[:n]
        lambda_c = x_3[n:]
        t = line_search(f, df, g, x, eta, d, d_c, bar_lambda, nu, m)
        x = x + t * (d + t * d_c)
        iter += 1
        print(f"{iter:2d} {t:1.10f} {x} {f(x):1.9f} {error:1.9f}")


In [36]:
f1 = lambda x: 2 * x[0]**2 + x[1]**2
df1 = lambda x: np.array([4 * x[0], 2 * x[1]])
g1 = lambda x: np.array([4 * x[0]**2 + x[1]**2 - 2,
                        4 * x[0] + x[1] + 1])
dg1 = lambda x: np.array([[8 * x[0], 2 * x[1]],
                         [4, 1]]).T
x_sol = [-2 / 9, -1 / 9]
xi = 0.5
eta = 0.5
phi = 1.5
nu = 0.7578
x0 = [-1 / 3, -1]
m = 2
np.set_printoptions(precision=10, suppress=True, linewidth=np.nan)
print(' i step t[i]\tx[i]\t\t\t\tf(x[i])\t  error')
FAIPA(f1, df1, g1, dg1, x0, 1e-5, xi, eta, phi, nu, m)


 i step t[i]	x[i]				f(x[i])	  error
 1 0.7578000000 [-0.5668679556 -0.3226345383] 0.746771603 0.516189128
 2 1.0000000000 [-0.5101415746 -0.0307644019] 0.521435301 0.246311045
 3 1.0000000000 [-0.3777167447 -0.1405983144] 0.305107765 0.146985209
 4 1.0000000000 [-0.2780588673 -0.1425685691] 0.174959264 0.062256766
 5 1.0000000000 [-0.2472159319 -0.1180142308] 0.136158793 0.020291669
 6 1.0000000000 [-0.2318944523 -0.1222740844] 0.122501026 0.010760202
 7 1.0000000000 [-0.2292182509 -0.1079082327] 0.116726200 0.012886384
 8 1.0000000000 [-0.2233680842 -0.1193070114] 0.114020765 0.012363752
 9 1.0000000000 [-0.2256665587 -0.1044596482] 0.112762610 0.014615036
10 1.0000000000 [-0.221106412  -0.1199784051] 0.112170909 0.015674865
11 1.0000000000 [-0.2252787316 -0.1021940434] 0.111944636 0.017949427
12 0.5742608400 [-0.2224492429 -0.1133566166] 0.111817054 0.019599082
13 1.0000000000 [-0.2231115194 -0.1092024957] 0.111482685 0.004074222
14 1.0000000000 [-0.2218350779 -0.1135584964] 0.11131

array([-0.2222227117, -0.1111091695])